# Import Libraries

- `numpy` for array operations

- `wordfreq` as data source for word frequencies in different languages

In [7]:
import wordfreq
import numpy as np

# Data Acquisition

Get raw frequent word data from the `wordfreq` library

In [8]:
# Define all languages to be studied, identified within the wordfreq library by two character strings
latin_language_keys = ["ca", # Catalan
                       "cs", # Czech
                       "da", # Danish
                       "nl", # Dutch
                       "en", # English
                       "fi", # Finnish
                       "fr", # French
                       "de", # German
                       "hu", # Hungarian
                       "is", # Icelandic
                       "it", # Italian
                       "lv", # Latvian
                       "lt", # Lithuanian
                       "nb", # Norwegian
                       "pl", # Polish
                       "pt", # Portuguese
                       "ro", # Romanian
                       "es", # Spanish
                       "sv", # Swedish
                       "tr"] # Turkish

In [9]:
# Get the 5000 most used words of each language, ordered by frequency
word_dict = {}
for key in latin_language_keys:
    word_dict[key] = wordfreq.top_n_list(key, 5000, ascii_only=False)

# Substring Extraction

Define a function for getting all substrings of a certain size from a string

In [10]:
def substrings_between(s, min_size, max_size):
    """
    Return all substrings of `s` whose length is between `min_size` and `max_size` (inclusive).
    """
    max_size = min(max_size, len(s))
    min_size = max(1, min_size)  # ensure at least 1
    
    out = []
    for L in range(min_size, max_size + 1):
        for i in range(len(s) - L + 1):
            out.append(s[i:i+L])
    return out

Next, Build up a dictionary where each key is a unique substring and each value is an array representing the counts of that substring in each language, index in the same order as in `latin_language_keys`. Only consider substrings between 1 and 5 characters.

**For Example**: if the only languages being studied were 'en', 'fr', and 'de' and the substring 'tch' appeared twice in English, four times in french, and ten times in german, the entry would be:

{"tch": [2, 4, 10]}

In [11]:
# Initialize dict to store results
substring_dict = {}

# Loop over all languages
for i in range(len(latin_language_keys)):
    # Get word list for this language
    key = latin_language_keys[i]
    word_list = word_dict[key]

    # Loop over each word
    for word in word_list:
        # Get all substrings between 1 and 5 characters
        substrings = substrings_between(word, min_size=1, max_size=5)

        # Loop over each substring
        for substring in substrings:
            # If substring was never seen in any previous word, initialize a count list for it
            if substring not in substring_dict.keys():
                substring_dict[substring] = [0] * len(latin_language_keys)
            # Increment the proper element in the count list for this substring by 1
            substring_dict[substring][i] += 1

Now that all substrings that exist in the data have been identified and counted by language, convert `substring_dict` into a numpy array `count array` where `count_array[i][j]` is the number of occurences of the ith substring in the jth language.

In [12]:
all_substrings = np.array(list(substring_dict.keys()))
count_array = np.array([substring_dict[s] for s in all_substrings])
count_array.shape

(182319, 20)

There are 182,319 unique substrings present across the 20 studied languages.

# Which Substrings Most Strongly Identify Each Language?

Let $L = the~set~of~all~studied~languages$

Let $S = the~set~of~all~identified~substrings$.

In order to determine what substrings $s\in S$ are most useful for identifying a particular language $l\in L$, the quantity to be maximized is the likelihood ratio:

$$LR_{s,l} = \frac{P(s|l)}{P(s|\neg l)}$$

This can be interpreted  as **"The ratio of the probability that the substring appears in language l aganist the probability that it appears in any other language"** to answer the question **"How much more likely is this substring to appear in language l that to appear in any other language?"**

To calculate these probabilities let:
* $c_l(s)$ be the count of substring $s$ in language $l$
* $c_{\neg l}(s)$ be the count of substring $s$ in all studied languages but $l$
* $N_l$ be the total number of substrings present in language $l$
* $N_{\neg l}$ be the total number of substrings present in all studied languages but $l$

then:

$$P(s|l) = \frac{c_l(s)}{N_l},~P(s|\neg l)=\frac{c_{\neg l}(s)}{N_{\neg l}}$$

Based on these equations, it is possible for $P(s|\neg l)$ to equal $0$ if a particular substring $s$ is only present in language $l$ and in no other languages, this causes the likelihood ratio to have $0$ in the denominator, leading to a $LR=\infty$. This would technically lead to the identification of substrings that uniquely identify a language, but they might not be very common substrings. A substring could only ever appear once in a single language and zero times in other languages and still have an infinite likelihood ratio. 

To address this issue, [additive smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) is applied to both conditional probabilities. This is a technique designed to smooth out count data to avoid zero counts like in this situation. A small constant $\alpha$ is added to the numerator and $\alpha|S|$ is added to the denominator, with $|S|$ being the total number of substrings in the set $S$. This has the effect of adding a small probability that a substring is possible in a language that is has never been observed in.

Therefore the final equations are:

$$P'(s|l) = \frac{c_l(s) + \alpha}{N_l + \alpha|S|},~P'(s|\neg l)=\frac{c_{\neg l}(s) + \alpha}{N_{\neg l} + \alpha|S|}$$

Therefore the final likelihood ratio to be maximized is:

$$LR_{s,l} = \frac{P'(s|l)}{P'(s|\neg l)} = \frac{(c_l(s) + \alpha)(N_{\neg l} + \alpha|S|)}{(N_l + \alpha|S|)(c_{\neg l}(s) + \alpha)}$$

This likelihood ratio is calculated below for each $s, l$ combination using $\alpha=0.5$

In [15]:
alpha = 0.1

# Initialize Results Array
likelihood_ratio_array = np.empty(count_array.shape)

# Loop over all languages
for lang_idx in range(len(latin_language_keys)):
    # Calculate total number of substrings in this language and not in this language
    total_substrings_lang = np.sum(count_array[:, lang_idx])
    total_substrings_other = np.sum(np.delete(count_array, lang_idx, axis=1))

    # loop over all substrings
    for substr_idx in range(len(all_substrings)):
        substring_counts = count_array[substr_idx]

        # Get number of times this substring appeared in this language and in all other languages
        count_in_lang = substring_counts[lang_idx]
        count_in_other = np.sum(np.delete(substring_counts, lang_idx))

        # Calculate likelihood ratio for this language/substring pair
        likelihood_ratio_array[substr_idx][lang_idx] = ((count_in_lang + alpha) / (total_substrings_lang + alpha * len(all_substrings))) / ((count_in_other + alpha) / (total_substrings_other + alpha * len(all_substrings)))


# Results: Most Distinctive Substrings per Language

In [16]:
log_likelihood_ratio_array = np.log10(likelihood_ratio_array)

# Loop over all languages
for i in range(len(latin_language_keys)):
    # Get array of substring sorted in descending order by likelihood ratio for this language
    indices_sorted = np.argsort(log_likelihood_ratio_array[:,i])[::-1]

    substrings_sorted = all_substrings[indices_sorted]
    likelihood_ratios_sorted = log_likelihood_ratio_array[:,i][indices_sorted]

    print(f"Language Code: {latin_language_keys[i]}")

    # Report first 5 most likely substrings for this language and the corresponding likelihood ratio
    for j in range(5):
        print(f"{substrings_sorted[j]}, log(LR)={likelihood_ratios_sorted[j]:.2f}")

    # Spacer
    print("-" * 40)

Language Code: ca
ènc, log(LR)=3.89
ènci, log(LR)=3.87
cions, log(LR)=3.82
ència, log(LR)=3.78
·l, log(LR)=3.63
----------------------------------------
Language Code: cs
ě, log(LR)=5.03
ř, log(LR)=4.83
ně, log(LR)=4.54
ů, log(LR)=4.48
ře, log(LR)=4.45
----------------------------------------
Language Code: da
øj, log(LR)=3.70
æng, log(LR)=3.65
søg, log(LR)=3.61
skab, log(LR)=3.55
øge, log(LR)=3.55
----------------------------------------
Language Code: nl
ijk, log(LR)=4.38
lijk, log(LR)=4.32
elijk, log(LR)=4.16
voor, log(LR)=3.91
ijke, log(LR)=3.91
----------------------------------------
Language Code: en
ally, log(LR)=3.67
tly, log(LR)=3.52
ough, log(LR)=3.41
ying, log(LR)=3.41
cted, log(LR)=3.38
----------------------------------------
Language Code: fi
ää, log(LR)=4.61
ään, log(LR)=4.19
tää, log(LR)=4.13
ssä, log(LR)=3.99
llä, log(LR)=3.99
----------------------------------------
Language Code: fr
êt, log(LR)=3.69
eux, log(LR)=3.64
rése, log(LR)=3.59
dép, log(LR)=3.54
prése, log(L